In [77]:
#import Libraries
import pandas as pd
import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import MonthEnd

In [23]:
#imprt sale record
schedule_path='sub_sales_m1.txt'

import_sales=pd.read_csv(schedule_path)

#update column data to datetime
import_sales.loc[:,['schedule_month','sales_month','amrt_start_month']]=import_sales.loc[:,['schedule_month','sales_month','amrt_start_month']].astype('datetime64[ns]')

In [24]:
import_sales['amrt_period']=import_sales['amrt_period'].astype('float64')

In [25]:
import_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   schedule_month    5 non-null      datetime64[ns]
 1   sales_month       5 non-null      datetime64[ns]
 2   source            5 non-null      object        
 3   material#         5 non-null      object        
 4   sales_amt         5 non-null      float64       
 5   amrt_start_month  5 non-null      datetime64[ns]
 6   amrt_period       5 non-null      float64       
 7   amrt_factor       5 non-null      float64       
dtypes: datetime64[ns](3), float64(3), object(2)
memory usage: 448.0+ bytes


In [26]:
import_sales

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor
0,2022-06-30,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5
1,2022-06-30,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5
2,2022-06-30,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5
3,2022-06-30,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5
4,2022-06-30,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5


In [27]:
#set beginning bal
amort_schedule=import_sales.copy()
amort_schedule['beg_bal']=amort_schedule['sales_amt']
amort_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal
0,2022-06-30,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99
1,2022-06-30,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99
2,2022-06-30,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99
3,2022-06-30,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99
4,2022-06-30,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99


In [78]:
def import_data(file_path):
    #import data from path
    import_sales=pd.read_csv(file_path)
    #update column data to datetime
    import_sales.loc[:,['schedule_month','sales_month','amrt_start_month']]=import_sales.loc[:,['schedule_month','sales_month','amrt_start_month']].astype('datetime64[ns]')
    #get last date of month
    import_sales['schedule_month']=import_sales['schedule_month']+MonthEnd(0)
    
    #update datatype
    import_sales['amrt_period']=import_sales['amrt_period'].astype('float64')
    #set beginning bal
    import_sales['beg_bal']=import_sales['sales_amt']
    return import_sales
    
    
    

In [79]:
schedule_path='sub_sales_m1.txt'

jun22_schedule=import_data(schedule_path)
jun22_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal
0,2022-06-30,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99
1,2022-06-30,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99
2,2022-06-30,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99
3,2022-06-30,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99
4,2022-06-30,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99


In [7]:
#calculate remainder numbers of month


amort_schedule['remainder']=amort_schedule['amrt_period']-(amort_schedule['schedule_month'].dt.year-amort_schedule['amrt_start_month'].dt.year)*12-(amort_schedule['schedule_month'].dt.month-amort_schedule['amrt_start_month'].dt.month)-amort_schedule['amrt_factor']

In [8]:
amort_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal,remainder
0,2022-06-30,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99,0.5
1,2022-06-30,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99,11.5
2,2022-06-30,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99,5.5
3,2022-06-30,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99,4.5
4,2022-06-30,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99,-0.5


In [9]:
#update remainder for negative value
amort_schedule.loc[amort_schedule['remainder']<0,'remainder']=0

In [18]:

#add end bal column
amort_schedule['end_bal']=round(amort_schedule['remainder']*amort_schedule['sales_amt']/amort_schedule['amrt_period'],2)

In [20]:
#add monthly amort column
amort_schedule['monthly_amort']=amort_schedule['beg_bal']-amort_schedule['end_bal']

In [21]:
amort_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal,remainder,end_bal,monthly_amort
0,2022-06-30,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99,0.5,6.50,6.49
1,2022-06-30,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99,11.5,57.49,2.50
2,2022-06-30,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99,5.5,39.41,3.58
3,2022-06-30,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99,4.5,29.99,10.00
4,2022-06-30,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99,0.0,0.00,9.99


In [80]:
#function to render remainder, end bal and monthly amort

def amort_cal(df):
    #calculate remainder numbers of month
    df['remainder']=df['amrt_period']-(df['schedule_month'].dt.year-df['amrt_start_month'].dt.year)*12-(df['schedule_month'].dt.month-df['amrt_start_month'].dt.month)-df['amrt_factor']
    #update remainder for negative value
    df.loc[df['remainder']<0,'remainder']=0
    #add end bal column
    df['end_bal']=round(df['remainder']*df['sales_amt']/df['amrt_period'],2)
    #add monthly amort column
    df['monthly_amort']=df['beg_bal']-df['end_bal']
    return df


    
    

In [81]:
amort_schedule_final=amort_cal(jun22_schedule)

In [82]:
jun22_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal,remainder,end_bal,monthly_amort
0,2022-06-30,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99,0.5,6.50,6.49
1,2022-06-30,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99,11.5,57.49,2.50
2,2022-06-30,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99,5.5,39.41,3.58
3,2022-06-30,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99,4.5,29.99,10.00
4,2022-06-30,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99,0.0,0.00,9.99


In [83]:
#create next month schedule

jul22_schedule=pd.concat([jun22_schedule.iloc[:,0:8],jun22_schedule['end_bal']],axis=1)
jul22_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,end_bal
0,2022-06-30,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,6.50
1,2022-06-30,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,57.49
2,2022-06-30,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,39.41
3,2022-06-30,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,29.99
4,2022-06-30,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,0.00


In [84]:
jul22_schedule['schedule_month']=jul22_schedule['schedule_month']+MonthEnd(1)

In [85]:
jul22_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,end_bal
0,2022-07-31,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,6.50
1,2022-07-31,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,57.49
2,2022-07-31,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,39.41
3,2022-07-31,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,29.99
4,2022-07-31,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,0.00


In [86]:
jul22_schedule.rename(columns={'end_bal':'beg_bal'},inplace=True)

In [87]:
jul22_schedule

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal
0,2022-07-31,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,6.50
1,2022-07-31,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,57.49
2,2022-07-31,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,39.41
3,2022-07-31,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,29.99
4,2022-07-31,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,0.00


In [88]:
schedule_path='sub_sales_m2.txt'

jul22_import=import_data(schedule_path)
jul22_import

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal
0,2022-07-31,2022-07-30,walmart,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99
1,2022-07-31,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99
2,2022-07-31,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99
3,2022-07-31,2022-05-31,google,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99
4,2022-07-31,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99


In [89]:
jul22_schedule_complete=pd.concat([jul22_schedule,jul22_import],axis=0,ignore_index=True)

In [90]:
jul22_schedule_complete

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal
0,2022-07-31,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,6.50
1,2022-07-31,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,57.49
2,2022-07-31,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,39.41
3,2022-07-31,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,29.99
4,2022-07-31,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,0.00
5,2022-07-31,2022-07-30,walmart,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99
6,2022-07-31,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99
7,2022-07-31,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99
8,2022-07-31,2022-05-31,google,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99
9,2022-07-31,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99


In [91]:
jul22_schedule_final=amort_cal(jul22_schedule_complete)

In [92]:
jul22_schedule_final

,schedule_month,sales_month,source,material#,sales_amt,amrt_start_month,amrt_period,amrt_factor,beg_bal,remainder,end_bal,monthly_amort
0,2022-07-31,2022-06-30,Costco,netflix_1mo,12.99,2022-06-30,1.0,0.5,6.50,0.0,0.00,6.50
1,2022-07-31,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,57.49,10.5,52.49,5.00
2,2022-07-31,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,39.41,4.5,32.24,7.17
3,2022-07-31,2022-05-31,Amazon,Amazon_6mo,39.99,2022-05-31,6.0,0.5,29.99,3.5,23.33,6.66
4,2022-07-31,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,0.00,0.0,0.00,0.00
5,2022-07-31,2022-07-30,walmart,netflix_1mo,12.99,2022-06-30,1.0,0.5,12.99,0.0,0.00,12.99
6,2022-07-31,2022-06-30,netflix,netflix_12mo,59.99,2022-06-30,12.0,0.5,59.99,10.5,52.49,7.50
7,2022-07-31,2022-06-30,Target,netflix_6mo,42.99,2022-06-30,6.0,0.5,42.99,4.5,32.24,10.75
8,2022-07-31,2022-05-31,google,Amazon_6mo,39.99,2022-05-31,6.0,0.5,39.99,3.5,23.33,16.66
9,2022-07-31,2022-05-31,Amazon,hulu_1mo,9.99,2022-05-31,1.0,0.5,9.99,0.0,0.00,9.99


In [ ]:
#function to carry end bal/lines to beg balto of next schedule

def carryover(df):
    
    #copy from existing schedule
    carryover_df=pd.concat([df.iloc[:,0:8],df['end_bal']],axis=1)
    #update schedule month
    carryover_df['schedule_month']=carryover_df['schedule_month']+MonthEnd(1)
    #rename columns
    carryover_df.rename(columns={'end_bal':'beg_bal'},inplace=True)
    return carryover_df
    
    